## Import Libraries

In [1]:
from os import path
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio
from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import phase, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## PAC Decoding - Low//High Encoding
### For ALL time points

In [7]:
from brainpipe.feat.utils._feat import _manageWindow
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

# ============================================================================================
nperm = 1000
conds,steps, subjects = ['low','high'],['odor'],['CHAF']#['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
color_codes = ['darkorange','blue']
# ============================================================================================
st = study('Olfacto')
path_phase = path.join(st.path, 'feature/3_PrefPhase_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/3_Classif_PrefPha_Encoding_EpiPerf_LowHigh_1000perm/')
# ============================================================================================
          
for su, step in product(subjects,steps):
    pfdpha_list = []
    #=========================== Load phase files (nfreq, nelec, nwin, ntrial) =================================    
    mat1 = np.load(path.join(path_phase, su+'_'+step+'_'+conds[0]+'_bipo_sel_physFT_prefpha.npz'))
    names, channels, pha_names = mat1['Mai_RL'], mat1['channels'],mat1['fname']
    ampl_names = ['beta','low-gamma','high-gamma']
    sWidth, sStep = 358, 51
    #PrfPha arrays (nfreqs x nelecs x npts x ntrials)
    pfdpha_list.append(mat1['pfd_meqn'][:,:,:,22:47,:])
    nampl,npha,nelecs,npts,ntrials = mat1['pfd_meqn'][:,:,:,22:47,:].shape
    _, time = _manageWindow(3584, width=sWidth, step=sStep)
    time = (np.array(time[22:47])/512)-3
    mat2name = path.join(path_phase, su+'_'+step+'_'+conds[1]+'_bipo_sel_physFT_prefpha.npz')
    if path.isfile(mat2name) == True:
        pfdpha_list.append(np.load(mat2name)['pfd_meqn'][:,:,:,22:47,:])
        print (su, 'pfdpha shape: ', [pfdpha.shape for pfdpha in pfdpha_list])

        # =========================== Select phase for 1 elec 1 freq =================================                 
        for elec_num in range(nelecs):
            for ampl,pha in product(range(nampl),range(npha)):
                elec, elec_label = channels[elec_num], names[elec_num]
                pha_name, ampl_name = pha_names[pha], ampl_names[ampl]
                
                #Filenames to save
                name_auc = (save_path+str(ampl)+'_'+ampl_name+'/'+str(pha)+'_'+pha_name+'/auc/'+su +'_auc_'+conds[0].capitalize()+'_'+conds[1].capitalize()+'_'+str(elec_label)+'_('+str(elec_num)+').npy')
                name_perm = (save_path+str(ampl)+'_'+ampl_name+'/'+str(pha)+'_'+pha_name+'/auc/'+su +'_perm_'+conds[0].capitalize()+'_'+conds[1].capitalize()+'_'+str(elec_label)+'_('+str(elec_num)+').npy')
                
                if path.exists(name_auc):
                    print(su,elec_num,ampl,pha,'already computed')
                else:
                    pfdpha_data_elec = []
                    for i,pfdpha in enumerate(pfdpha_list):
                        pfdpha_data_elec.append(pfdpha[ampl,pha,elec_num].swapaxes(0,1))
                    nwin = pfdpha.shape[1] #(ntrials x nwins)
                    print('--» processing',su, 'elec', elec_num,'/',nelecs, 'ampl',ampl_name,'pha',pha_name)
                    print ('elec ', elec, 'elec_label ', elec_label,'data',[pfdpha.shape for pfdpha in pfdpha_data_elec])
            # =============================  Classification Computation ============================================================           
                   # create a data matrix, concatenate along the trial dimension
                    x = np.concatenate(pfdpha_data_elec, axis=0)
                    print ('Size of the concatenated data: ','Number time windows : ', x.shape[1])
                    y = np.hstack([np.array([i]*len(pfdpha)) for i, pfdpha in enumerate(pfdpha_data_elec)])
                    print ('Size of label for classif: ', len(y))

                    auc = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        score_rep = []
                        for i in range(10):
                            k = 5
                            if su == 'CHAF':
                                k = 3
                            skf = SKFold(n_splits=k, random_state=None, shuffle=True)
                            skf.get_n_splits(X, y)
                            score_cv = []
                            for train_index, test_index in skf.split(X, y):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y[train_index], y[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                            score_rep.append(np.mean(score_cv))
                        score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
                        auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
                    auc = np.swapaxes(auc,0,1)

                    perm_scores = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        perm_rep = []
                        for perm in range(nperm):
                            y_perm = y[permutation(len(y))]
                            score_cv = []
                            for train_index, test_index in skf.split(X, y_perm):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y_perm[train_index], y_perm[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                            perm_rep.append(np.mean(score_cv))
                        perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
                        perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
                    perm_scores = np.swapaxes(perm_scores,0,1)
                    th_0_05_perm = perm_pvalue2level(perm_scores, p=0.05, maxst=True)
                    th_0_01_perm = perm_pvalue2level(perm_scores, p=0.01, maxst=True)
                    print('th_perm : ', th_0_05_perm[0], th_0_01_perm[0], 'auc_mean', auc.mean())

        # ============================== PLOT phase ANALYSIS + STATS & DECODING ACCURACY ===================================================                
                    xfmt = ScalarFormatter(useMathText=True)
                    xfmt.set_powerlimits((0,3))
                    fig = plt.figure(1,figsize=(7,7))
                    title = 'Pfdpha for '+su+' '+conds[0]+' vs '+conds[1]+' '+str(elec)+' '+str(elec_label)+' ('+str(elec_num)+')'
                    fig.suptitle(title, fontsize=12)

                    # Plot the phase + STATS
                    plt.subplot(211)      
                    BorderPlot(time, x, y=y, kind='sem', alpha=0.2, color=color_codes,linewidth=2, 
                               ncol=1, xlabel='Time (s)',ylabel = r'pfdpha', legend=conds)
                    rmaxis(plt.gca(), ['right', 'top'])
                    plt.legend(loc=0, handletextpad=0.1, frameon=False)
                    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

                    # Plot DA for the phase
                    plt.subplot(212)
                    BorderPlot(time, auc, color='b', kind='sd',xlabel='Time (s)', ylim=[0.4,1.], ylabel='Decoding accuracy (%)',linewidth=2, alpha=0.3)
                    rmaxis(plt.gca(), ['right', 'top'])
                    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
                    plt.plot(time, th_0_05_perm*np.ones(len(time)), '--', color='r', linewidth=2)
                    plot_name = (save_path+str(ampl)+'_'+ampl_name+'/'+str(pha)+'_'+pha_name+'/fig/'+su +'_Pac_'+conds[0]+'_'+conds[1]+'_'+str(elec)+'_'+str(elec_label)+'_('+str(elec_num)+').png')            

                    #Save plots
                    np.save(name_auc, auc)
                    np.save(name_perm, perm_scores)
                    plt.savefig(plot_name, dpi=300, bbox_inches='tight')
                    plt.clf()
                    plt.close()
                    del X, auc, pfdpha_data_elec
        del pfdpha_list

    else: 
        print(su,'pas de condition ',conds[1].capitalize())

-> Olfacto loaded
CHAF pfdpha shape:  [(3, 4, 58, 25, 6), (3, 4, 58, 25, 15)]
--» processing CHAF elec 0 / 58 ampl beta pha VLFC
elec  c'2-c'1 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21


/home/karim/anaconda3/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:402: RuntimeWarning: invalid value encountered in true_divide
  S**2))[:self._max_components]


th_perm :  0.633333333333 0.8 auc_mean 0.4888
not legend
--» processing CHAF elec 0 / 58 ampl beta pha delta
elec  c'2-c'1 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.633333333333 0.8 auc_mean 0.490133333333
not legend
--» processing CHAF elec 0 / 58 ampl beta pha theta
elec  c'2-c'1 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.633333333333 0.8 auc_mean 0.487866666667
not legend
--» processing CHAF elec 0 / 58 ampl beta pha alpha
elec  c'2-c'1 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
Size of label for classif:  21
th_perm :  0.633333333333 0.8 auc_mean 0.529
not legend
--» processing CHAF elec 0 / 58 ampl low-gamma pha VLFC
elec  c'2-c'1 elec_label  SFG data [(6, 25), (15, 25)]
Size of the concatenated data:  Number time windows :  25
